In [5]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
from requests_html import HTMLSession
import os


from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
# import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from enhanced_eps_extraction import extract_eps_with_requests_html_chunked 



chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--disable-dev-shm-usage")  # Helps in Docker environments

from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        # user_agent='your-custom-user-agent',  # Set custom user agent
        languages=["en-US", "en"],  # Set languages
        vendor="Mozilla",  # Set vendor
        platform="Win32",  # Set platform
        webgl_vendor="Mozilla",  # Set WebGL vendor
        renderer="Mozilla",  # Set WebGL renderer
        fix_hairline=True)  # Fix hairline issue in headless mode
driver.get('https://www.nseindia.com/market-data/52-week-low-equity-market')
cookies = driver.get_cookies()
time.sleep(2)


cookie = ""
for item in driver.get_cookies():
    cookie += item['name'] + "=" + item['value'] + "; "

In [6]:
# driver.quit()

<h1>Indexes and stock of indexes

In [7]:


# # get all indices
# url = "https://www.nseindia.com/api/allIndices"

# payload = {}
# headers = {
#   'accept': '*/*',
#   'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
#   'priority': 'u=1, i',
  
#   'referer': 'https://www.nseindia.com/market-data/live-market-indices',
#   'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
#   'sec-ch-ua-mobile': '?0',
#   'sec-ch-ua-platform': '"macOS"',
#   'sec-fetch-dest': 'empty',
#   'sec-fetch-mode': 'cors',
#   'sec-fetch-site': 'same-origin',
#   'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
#   'Cookie': cookie
# }

# response = requests.request("GET", url, headers=headers, data=payload)
# indices = pd.DataFrame(json.loads(response.text)['data'])
# index = indices['index'].tolist()
# index = [index.replace(" ", "%20") for index in index]



In [8]:
# total_data = pd.DataFrame()
# # get indices data
# for indx in tqdm(index):
#   url = f"https://www.nseindia.com/api/equity-stockIndices?index={indx}"

#   payload = {}
#   headers = {
#     'accept': '*/*',
#     'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
#     'priority': 'u=1, i',
#     'referer': 'https://www.nseindia.com/market-data/live-equity-market?symbol=NIFTY%2050',
#     'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
#     'sec-fetch-dest': 'empty',
#     'sec-fetch-mode': 'cors',
#     'sec-fetch-site': 'same-origin',
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
#     'Cookie': cookie
#   }

#   response = requests.request("GET", url, headers=headers, data=payload)
#   if json.loads(response.text).get('data'):
#     total_data = pd.concat([total_data, pd.DataFrame(json.loads(response.text)['data'])[['symbol','identifier']]] , ignore_index=True)



In [ ]:
with open('selenium_utils.py') as f:
    code = f.read()
    exec(code)


INFO:__main__:Initialized SeleniumUtils with download_dir: /Users/admin/Desktop/stock_project
INFO:__main__:Found Chrome at: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.35/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Initialized SeleniumUtils with download_dir: /Users/admin/Desktop/stock_project
INFO:__main__:Found Chrome at: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
INFO:__main__:Using Chrome binary: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome


=== Chrome Installation Diagnosis ===
Operating System: Darwin
Architecture: 64bit
Chrome Found: True
Chrome Path: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
ChromeDriver Available: True
ChromeDriver Path: /Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.35/chromedriver-mac-arm64/chromedriver


INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.35/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Chrome driver created with automatic ChromeDriver management
INFO:__main__:Chrome driver created successfully
INFO:__main__:Successfully obtained 8 cookies from https://www.nseindia.com/


Got cookies: _abck=306639163E31E86030E4F1A02B97B3A6~-1~YAAQj88uF990sGOXAQAAB79RmQ6gJhdciAxDLFf1Bd7Npw8tiPFvn6gRgO...


In [25]:
total_data = pd.read_csv('EQUITY_L.csv',dtype=str)
total_data.head(5)
total_data.columns = [col.lower() for col in total_data.columns]


In [26]:
# filter index names from stocks
symbol = list(set(total_data['symbol'].tolist()))


<h1> Multithreading Annual reports


In [ ]:
import concurrent.futures
import threading

# Thread-local storage for DataFrame
thread_local = threading.local()

def get_symbol_data(sym):
    """Function to fetch data for a single symbol"""
    url = f"https://www.nseindia.com/api/corporates-financial-results?index=equities&symbol={sym}&period=Annual"
    
    payload = {}
    headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'priority': 'u=1, i',
        'referer': f'https://www.nseindia.com/companies-listing/corporate-filings-financial-results?symbol={sym}&tabIndex=equity',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'Cookie': cookie
    }
    
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        annual_data_df = pd.DataFrame(json.loads(response.text))
        return annual_data_df
    except Exception as e:
        print(f"Error fetching data for {sym}: {e}")
        return pd.DataFrame()

# Use ThreadPoolExecutor for parallel processing
reports = pd.DataFrame()
max_workers = 5  # Adjust based on your system and API limits

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_symbol = {executor.submit(get_symbol_data, sym): sym for sym in symbol}
    
    # Collect results with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_symbol), total=len(symbol)):
        symbol_name = future_to_symbol[future]
        try:
            result_df = future.result()
            if not result_df.empty:
                reports = pd.concat([reports, result_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {symbol_name}: {e}")


In [28]:
driver.quit()

In [ ]:
len(set(total_data['symbol'].unique().tolist()) - set(reports['symbol'].unique().tolist()))

In [29]:
reports.to_excel('reports.xlsx', index=False)

In [ ]:
reports

In [30]:
# audited by external auditors

audited_df = reports[reports['audited']=='Audited'].copy(deep=True)
# cumulative means collection of all (q1+q2+q3+q4) 

audited_df = audited_df[audited_df['cumulative']=='Cumulative']

In [31]:
control_df = audited_df[['symbol', 'companyName', 'fromDate', 'toDate',  'xbrl',
        'consolidated']].copy(deep=True)

control_df['fromDate'] = pd.to_datetime(control_df['fromDate'], dayfirst=True).dt.strftime('%Y-%m-%d')
control_df['toDate'] = pd.to_datetime(control_df['toDate'], dayfirst=True).dt.strftime('%Y-%m-%d')

In [ ]:
control_df.head(5)

In [33]:
control_df = control_df[['symbol','xbrl','consolidated']]

In [34]:
reports_cut = reports[['symbol','xbrl','consolidated']].copy(deep=True)
grouped_df = reports_cut.groupby(['symbol', 'consolidated'])['xbrl'].agg(list).reset_index()

# clearing invalid urls
grouped_df['xbrl'] = grouped_df['xbrl'].apply(lambda x: [item for item in x if item.rsplit('/',1)[-1]!='-'])

In [35]:
grouped_df.to_excel('grouped_df.xlsx', index=False)

In [ ]:
grouped_df

In [ ]:
result = extract_eps_with_requests_html_chunked(grouped_df , max_workers=5)

In [16]:
grouped_df.to_excel('grouped_df_eps.xlsx', index=False)

<h1>Computing the avg eps , removing None type eps(failed to extract)

In [17]:
grouped_df['avg_eps'] = grouped_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]
    ) / len(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]
    ) if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]) > 0 else "",
    axis=1
)

In [18]:
grouped_df['total_eps'] = grouped_df.apply(
    lambda x: len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float, str))]),
    axis=1
)

In [19]:
grouped_df['3Yr_avg_eps'] = grouped_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))][:3]
    ) / 3 if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]) >= 3 else "", 
    axis=1
)

In [20]:
seven_and_above = grouped_df[grouped_df['total_eps']>=7].copy(deep=True)
three = grouped_df[(grouped_df['3Yr_avg_eps']!="")].copy(deep=True)
three = three[(three['3Yr_avg_eps']>0)].copy(deep=True)


In [21]:
three['fifftee_times']  = ""
three['fifftee_times'] = three.apply(lambda x: 15*x['3Yr_avg_eps'],axis=1)


three.reset_index(drop=True,inplace=True)
three['cmp'] = ""

In [22]:
# def get_price(symbol):
#     url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

#     payload = {}
#     headers = {
#     'accept': '*/*',
#     'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
#     'cookie': cookie,
#     'priority': 'u=1, i',
#     'referer': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE',
#     'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
#     'sec-fetch-dest': 'empty',
#     'sec-fetch-mode': 'cors',
#     'sec-fetch-site': 'same-origin',
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
#     }

#     response = requests.request("GET", url, headers=headers, data=payload)

#     return json.loads(response.text)['priceInfo']['lastPrice']


# for i in tqdm(range(three.shape[0])):
#     three.loc[i,'cmp'] = get_price(three.loc[i,'symbol'])


In [ ]:
# Multithreaded version of price fetching - Copy this into your notebook cell

def get_price_with_error_handling(symbol):
    """Get price for a symbol with error handling"""
    try:
        url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

        payload = {}
        headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cookie': cookie,
        'priority': 'u=1, i',
        'referer': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        }

        response = requests.request("GET", url, headers=headers, data=payload, timeout=30)
        return json.loads(response.text)['priceInfo']['lastPrice']
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return None

# Get list of symbols and their indices
symbols_with_indices = [(i, symbol) for i, symbol in enumerate(three['symbol'])]

# Use ThreadPoolExecutor for parallel price fetching
max_workers = 10  # Adjust based on your system and API limits
prices_dict = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_index = {executor.submit(get_price_with_error_handling, symbol): (i, symbol) 
                      for i, symbol in symbols_with_indices}
    
    # Collect results with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_index), 
                      total=len(symbols_with_indices),
                      desc="Fetching prices (multithreaded)"):
        index, symbol = future_to_index[future]
        try:
            price = future.result()
            prices_dict[index] = price
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            prices_dict[index] = None

# Update the DataFrame with fetched prices
for index, price in prices_dict.items():
    three.loc[index, 'cmp'] = price

print(f"Completed fetching prices for {len(prices_dict)} symbols") 

In [25]:
three['cmp'] = three['cmp'].apply(lambda x: float(x))

In [ ]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'],inplace=True)

In [ ]:
three[three['fifftee_times']>three['cmp']].sort_values(by=['cmp'])